In [ ]:
!pip install geopandas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches
import geopandas as gpd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# block_name = 'Mohanpur'
block_name = 'Pindwara'
shp_path =  f'/content/drive/MyDrive/drought_modisv2/{block_name}.shp'
gdf = gpd.read_file(shp_path)
# print(gdf.head(10))
block_name = 'Pindwara10'
filepathPrefix = f'/content/drive/MyDrive/drought_modis_mask/{block_name}_'

vcibins = [-1.0, 40.0, 60.0, 100.0]
vcilabels = ['poor', 'fair', 'good']

maibins = [-1.0, 25.0, 50.0, 100.0]
mailabels = ['severe', 'moderate', 'mild']

casbins = [-1.0, 33.3, 66.6, 100.0]
caslabels = ['poor', 'fair', 'good']

spibins = [-1000.0, -2.0, -1.5, -1.0, 0.0, 1.0, 1.5, 2.0, 1000.0 ]
spilabels = ['extremelyDry', 'severelyDry', 'moderatelyDry', 'mildlyDry', 'mildlyWet', 'moderatelyWet', 'severelyWet', 'extremelyWet' ]


for year in range(2003,2023):
  finpath = filepathPrefix + str(year) + ".csv"
  df = pd.read_csv(finpath)

  ##Meterological Drought START
  mDrought=0
  mDroughtCols = [col for col in df.columns if 'meteorological_drought' in col]

  df[f'mDrought_{year}'] = df[mDroughtCols].sum(axis=1)

  ##DrySPells START
  drySpells=0
  drySpellsCols = [col for col in df.columns if 'dryspell' in col]
  df[f'drySpells_{year}'] = df[drySpellsCols].sum(axis=1)

  ##Cropping Area Sown in Kharif
  df[f'cas_{year}_mode']=0
  for index, row in df.iterrows():
    val = df.at[index,f'percent_of_area_cropped_kharif_{year}']
    if(val<=33.3):
      df.at[index ,f'cas_{year}_mode']=3
    elif (val<=66.6):
      df.at[index ,f'cas_{year}_mode']=2
    else:
      df.at[index ,f'cas_{year}_mode']=1

  ##VCI START
  vciCols = [col for col in df.columns if 'vci' in col]
  for l in vcilabels:
    df[f'vci_{year}_{l}']=0

  for index, row in df.iterrows():
      for col in vciCols:
          vci_value = row[col]
          category = pd.cut([vci_value], bins=vcibins, labels=vcilabels)[0]
          df.at[index, f'vci_{year}_{category}'] += 1

  df[f'vci_{year}_mode']=0
  for index, row in df.iterrows():
    templ=[]
    i=1
    for l in vcilabels[::-1]:
      templ.append([df.at[index , f'vci_{year}_{l}'],i])
      i+=1
    templ.sort()
    df.at[index ,f'vci_{year}_mode']=templ[-1][1]


  ##MAI START
  maiCols = [col for col in df.columns if 'mai' in col]
  for l in mailabels:
    df[f'mai_{year}_{l}']=0
  for index, row in df.iterrows():
      for col in maiCols:
          mai_value = row[col]
          category = pd.cut([mai_value], bins=maibins, labels=mailabels)[0]
          df.at[index, f'mai_{year}_{category}'] += 1

  df[f'mai_{year}_mode']=0
  for index, row in df.iterrows():
    templ=[]
    i=1
    for l in mailabels[::-1]:
      # print(l)
      templ.append([df.at[index , f'mai_{year}_{l}'],i])
      i+=1
    templ.sort()
    df.at[index ,f'mai_{year}_mode']=templ[-1][1]



  ##SPI START
  spiCols = [col for col in df.columns if 'spi' in col]
  for l in spilabels:
    df[f'spi_{year}_{l}']=0
  for index, row in df.iterrows():
      for col in spiCols:
          spi_value = row[col]
          category = pd.cut([spi_value], bins=spibins, labels=spilabels)[0]
          df.at[index, f'spi_{year}_{category}'] += 1

# ['extremelyDry', 'severelyDry', 'moderatelyDry', 'mildlyDry', 'mildlyWet', 'moderatelyWet', 'severelyWet', 'extremelyWet' ]
  df[f'spi_{year}_mode']=0
  for index, row in df.iterrows():
    templ=[]
    i=1
    for l in spilabels[::-1]:
      templ.append([df.at[index , f'spi_{year}_{l}'],i])
      i+=1
    templ.sort()
    df.at[index ,f'spi_{year}_mode']=templ[-1][1]


  # print(len(mDroughtCols))
  # print(len(drySpellsCols))
  # print(len(vciCols))
  # print(len(maiCols))
  # print(len(spiCols))
  gdf = pd.merge(gdf,df, on='uid', how='left')

# print(gdf.columns.tolist)

# print(gdf[['vci_2003_good','vci_2003_fair', 'vci_2003_poor', 'mDrought_2003', 'drySpells_2003',
#            'mai_2003_no', 'mai_2003_mild', 'mai_2003_moderate', 'mai_2003_severe',
#            'spi_2003_extremelyDry', 'spi_2003_severelyDry', 'spi_2003_moderatelyDry', 'spi_2003_mildlyDry',
#            'spi_2003_mildlyWet', 'spi_2003_moderatelyWet', 'spi_2003_severelyWet', 'spi_2003_extremelyWet'
#            ]])


<ipython-input-4-3ec580795148>:120: FutureWarning: Passing 'suffixes' which cause duplicate columns {'id_x', 'area_in_ha_x'} in the result is deprecated and will raise a MergeError in a future version.
  gdf = pd.merge(gdf,df, on='uid', how='left')
<ipython-input-4-3ec580795148>:120: FutureWarning: Passing 'suffixes' which cause duplicate columns {'.geo_x', 'system:index_x', 'kharif_croppable_sqkm_x'} in the result is deprecated and will raise a MergeError in a future version.
  gdf = pd.merge(gdf,df, on='uid', how='left')
<ipython-input-4-3ec580795148>:120: FutureWarning: Passing 'suffixes' which cause duplicate columns {'id_x', 'area_in_ha_x'} in the result is deprecated and will raise a MergeError in a future version.
  gdf = pd.merge(gdf,df, on='uid', how='left')
<ipython-input-4-3ec580795148>:120: FutureWarning: Passing 'suffixes' which cause duplicate columns {'.geo_x', 'system:index_x', 'kharif_croppable_sqkm_x'} in the result is deprecated and will raise a MergeError in a futur

In [ ]:
# Just for Creating CSV
gdf.to_csv(f'/content/drive/MyDrive/drought_modis_mask/{block_name}_allvar.csv', index=False)